In [ ]:
# Task 1: Torus Simulator and Braitenberg Vehicles

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML, display

"""
Simulating a Braitenberg vehicle over a number of steps
 - sim: TorusSimulator instance
 - robot: BraitenbergRobot instance
 - ctrl: dictionary of controller functions
 - steps: number of simulation steps
 Returns: trajectory array of shape (steps, 2)
"""
def simulate_braitenberg(sim, robot, ctrl, steps=200):
    traj = np.zeros((steps,2))
    for i in range(steps):
        traj[i] = robot.pos.copy()       # record current position
        sim.step(robot, ctrl)           # advance simulator one time step
    return traj

class TorusSimulator:
    """
    Simulating motion on a toroidal (wrap-around) 2D world.
    Attributes:
      W, H: world width and height
      dt: time step
      max_speed: maximum forward speed
      max_turn: maximum turn rate (radians per step)
    """
    def __init__(self, width=100, height=100, dt=0.1, max_speed=5.0, max_turn=np.pi/4):
        self.W, self.H, self.dt = width, height, dt
        self.max_speed, self.max_turn = max_speed, max_turn

    def wrap(self, p):
        #Wraping a 2D position p around torus boundaries
        return np.mod(p, [self.W, self.H])

    def step(self, r, ctrl):
        """
        Advance robot r by one time step using controller ctrl.
        - r.read_sensors() provides left/right sensor readings
        - ctrl['vl'], ctrl['vr'] map sensor values to wheel speeds
        - forward speed is average of wheels, turn is difference
        """
        sl, sr = r.read_sensors()                         # left/right sensor values
        vl = ctrl['vl'](sl, sr); vr = ctrl['vr'](sl, sr)  # wheel speeds
        # compute forward speed and clamp
        speed = np.clip((vl + vr) / 2, -self.max_speed, self.max_speed)
        # compute angular change and clamp
        dphi = np.clip(ctrl.get('c',1) * (vr - vl), -self.max_turn*self.dt, self.max_turn*self.dt)
        # update heading
        r.phi = (r.phi + dphi + np.pi) % (2*np.pi) - np.pi
        # update position and wrap around edges
        r.pos = self.wrap(r.pos + speed*self.dt*np.array([np.cos(r.phi), np.sin(r.phi)]))

class LightField:
    """
    Defining a toroidal light intensity field decreasing linearly with toroidal distance from a source.
    Attributes:
      src: (x,y) source position
      max_d: maximum toroidal distance (half-diagonal of world)
    """
    def __init__(self, src=(50,50), W=100, H=100):
        self.src = np.array(src)
        self.max_d = np.hypot(W/2, H/2)

    def intensity(self, p):
        """
        Computing light intensity at position p:
        1 at source, 0 at max toroidal distance.
        """
        delta = np.abs(p - self.src)
        # wrap differences for torus
        delta = np.minimum(delta, [self.max_d, self.max_d] - delta)
        d = np.hypot(*delta)
        return max(0, 1 - d/self.max_d)

class BraitenbergRobot:
    """
    Braitenberg vehicle with two light sensors.
    Attributes:
      pos: 2D position
      phi: heading angle
      sd: sensor distance from center
      sa: sensor angle offset
      field: LightField instance
    """
    def __init__(self, pos, phi, sensor_dist=5.0, sensor_ang=np.pi/6, field=None):
        self.pos = np.array(pos, float)
        self.phi = phi
        self.sd, self.sa = sensor_dist, sensor_ang
        self.field = field

    def read_sensors(self):
        #Returns light intensities at left and right sensor positions

        angles = [self.phi + self.sa, self.phi - self.sa]
        vals = []
        for a in angles:
            sample = self.pos + self.sd * np.array([np.cos(a), np.sin(a)])
            vals.append(self.field.intensity(sample))
        return vals

# Simulation parameters
dt, W, H, steps = 0.1, 100, 100, 200
field = LightField((50,50), W, H)

# initializing two simulator instances for each behavior
sim1 = TorusSimulator(W, H, dt)

# controller gains k and turning constant c
k, c = 5.0, 0.5
ctrl_aggr = {'vl': lambda sl,sr: k*sr, 'vr': lambda sl,sr: k*sl, 'c': c}  # cross-coupling
ctrl_fear  = {'vl': lambda sl,sr: k*sl, 'vr': lambda sl,sr: k*sr, 'c': c}  # direct coupling

# creating robots with starting positions and headings
r1 = BraitenbergRobot((20,20), 0.0, 5.0, np.pi/6, field)
r2 = BraitenbergRobot((80,20), np.pi, 5.0, np.pi/6, field)
# simulating trajectories
traj1 = simulate_braitenberg(sim1, r1, ctrl_aggr, steps)
traj2 = simulate_braitenberg(TorusSimulator(W,H,dt), r2, ctrl_fear, steps)

# Plot and save static light field
yy, xx = np.meshgrid(np.linspace(0,W,200), np.linspace(0,H,200))
II = np.vectorize(lambda x,y: field.intensity(np.array([x,y])))(xx, yy)
plt.figure(figsize=(6,6))
plt.imshow(II, extent=[0,W,0,H], origin='lower', cmap='viridis')  # show intensity map
plt.title('Light Intensity Field')
plt.colorbar(label='Intensity')
plt.xlabel('X'); plt.ylabel('Y')
plt.savefig('task1_light_field.png'); display(plt.gcf()); plt.close()

# Plot and save static trajectories
plt.figure(figsize=(6,6))
plt.plot(traj1[:,0], traj1[:,1], label='Aggressor')  # path for cross-coupled vehicle
plt.plot(traj2[:,0], traj2[:,1], label='Fear')      # path for direct-coupled vehicle
plt.title('Braitenberg Trajectories')
plt.xlabel('X'); plt.ylabel('Y')
plt.legend(loc='upper right')  # fixed legend position
plt.savefig('task1_traj.png', bbox_inches='tight'); display(plt.gcf()); plt.close()

# Animate trajectories
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim(0,W); ax.set_ylim(0,H)
line1, = ax.plot([], [], lw=2, label='Aggressor')
line2, = ax.plot([], [], lw=2, label='Fear')
ax.legend(loc='upper right')

def init():
    """Initialize empty lines for animation."""
    line1.set_data([], [])
    line2.set_data([], [])
    return line1, line2

def update(i):
    #Updating lines at frame i
    line1.set_data(traj1[:i,0], traj1[:i,1])
    line2.set_data(traj2[:i,0], traj2[:i,1])
    return line1, line2

anim = FuncAnimation(fig, update, frames=steps, init_func=init, blit=True)
anim.save('task1_anim.gif', writer=PillowWriter(fps=20))
plt.close()
display(HTML('<img src="task1_anim.gif" />'))
